In [1]:
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.mount_pyrosetta_install()
  import pyrosetta
  pyrosetta.init('-extra_res_fa google_drive/MyDrive/PyRosetta_Workshop/MK8.params -load_PDB_components false')



Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.1+release.00b79147e63-cp310-cp310-linux_x86_64.whl


PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -extra_res_fa google_drive/MyDrive/PyRosetta_Workshop/MK8.params -load_PDB_components false -database 

In [2]:

#Python
# from pyrosetta import *
# from pyrosetta.rosetta import *
# from pyrosetta.teaching import *

# #Core Includes
# from rosetta.core.kinematics import MoveMap
# from rosetta.core.kinematics import FoldTree
# from rosetta.core.pack.task import TaskFactory
# from rosetta.core.pack.task import operation
# from rosetta.core.simple_metrics import metrics
# from rosetta.core.select import residue_selector as selections
# from rosetta.core import select
# from rosetta.core.select.movemap import *

# #Protocol Includes
# from rosetta.protocols import minimization_packing as pack_min
# from rosetta.protocols import relax as rel
# from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
# from rosetta.protocols.antibody import *
# from rosetta.protocols.loops import *
# from rosetta.protocols.relax import FastRelax
# from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector, OrResidueSelector


In [3]:


mol2_ligand_path = '/content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/bla_ideal.mol2'
sdf_ligand_path= '/content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/bla_ideal.sdf'




In [4]:
#!/usr/bin/env python3

import pyrosetta
from pyrosetta import rosetta

DOCKSCRIPT = '''
<ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep"/>
        <ScoreFunction name="hard_rep" weights="ligand"/>
    </SCOREFXNS>

    <LIGAND_AREAS>
        <LigandArea name="inhibitor_dock_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_sc" chain="X" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
        <LigandArea name="inhibitor_final_bb" chain="X" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
    </LIGAND_AREAS>

    <INTERFACE_BUILDERS>
        <InterfaceBuilder name="side_chain_for_docking" ligand_areas="inhibitor_dock_sc"/>
        <InterfaceBuilder name="side_chain_for_final" ligand_areas="inhibitor_final_sc"/>
        <InterfaceBuilder name="backbone" ligand_areas="inhibitor_final_bb" extension_window="3"/>
    </INTERFACE_BUILDERS>

    <MOVEMAP_BUILDERS>
        <MoveMapBuilder name="docking" sc_interface="side_chain_for_docking" minimize_water="false"/>
        <MoveMapBuilder name="final" sc_interface="side_chain_for_final" bb_interface="backbone" minimize_water="false"/>
    </MOVEMAP_BUILDERS>

    <SCORINGGRIDS ligand_chain="X" width="35">
        <ClassicGrid grid_name="classic" weight="1.0"/>
    </SCORINGGRIDS>

    <MOVERS>
        <Transform name="transform" chain="X" box_size="7.0" move_distance="0.2" angle="20" cycles="500" repeats="1" temperature="1" initial_perturb="5.0"/>
        <HighResDocker name="high_res_docker" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking"/>
        <FinalMinimizer name="final" scorefxn="hard_rep" movemap_builder="final"/>
    </MOVERS>

    <PROTOCOLS>
        <Add mover_name="transform"/>
        <Add mover_name="high_res_docker"/>
        <Add mover_name="final"/>
    </PROTOCOLS>

</ROSETTASCRIPTS>
'''




In [5]:


def get_docking_mover():
    return rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(DOCKSCRIPT).get_mover("ParsedProtocol")

def load_pose(filename, paramsfile):
    fa_rts = rosetta.core.chemical.ChemicalManager.get_instance().residue_type_set("fa_standard")
    custom_rts = rosetta.core.chemical.PoseResidueTypeSet( fa_rts );
    custom_rts.add_base_residue_type(paramsfile)
    return pyrosetta.io.pose_from_file(custom_rts,filename)

if __name__ == "__main__":
    pyrosetta.init("-restore_pre_talaris_2013_behavior")

    dock = get_docking_mover()



PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -restore_pre_talaris_2013_behavior -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-778980096 seed_offset=0 real_seed=-778980096
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-778980096 RG_type=mt19937
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta

In [6]:
from pyrosetta.rosetta.core.chemical import ResidueTypeSet
from pyrosetta import init, pose_from_pdb
# init(options='-ignore_unrecognized_res')


params_path = '/content/google_drive/MyDrive/BLA_2.params'

pose_path = '/content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/LIGAND_PROTEIN_COMPLEX/relaxed_nir_iglu_BLA.pdb'





In [7]:
pose = load_pose(pose_path, params_path)



core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 74 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.077888 seconds.
core.import_pose.import_pose: File '/content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/LIGAND_PROTEIN_COMPLEX/relaxed_nir_iglu_BLA.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'BLA' from the PDB components dictionary for residue type 'pdb_BLA'
core.conformation.Conformation: Found disulfide between residues 84 209


In [8]:
pdb_info = pose.pdb_info()
print(pdb_info)

PDB file name: /content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/LIGAND_PROTEIN_COMPLEX/relaxed_nir_iglu_BLA.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0430    A 0001  -- 0430  |   0430 residues;    06864 atoms
0431 -- 0431    X 0001  -- 0001  |   0001 residues;    00077 atoms
                           TOTAL |   0431 residues;    06941 atoms



In [9]:
# Assuming 'pose' is your loaded pose
pdb_info = pose.pdb_info()
existing_chains = set()

# Iterate over residues and collect unique chain identifiers
for i in range(1, pose.total_residue() + 1):
    chain_id = pdb_info.chain(i)
    existing_chains.add(chain_id)

print("Existing chains:", existing_chains)

Existing chains: {'A', 'X'}


In [10]:
output_folder = '/content/google_drive/MyDrive/PyRosetta_Work/Ligand_Docked'




In [11]:
import os

for nstruct in range (1,1001):
  pose = load_pose(pose_path, params_path)
  dock.apply(pose)
  output_pdb_name = f'NIR_IGLU_BLA_LIGAND_DOCKED_{nstruct}.pdb'
  output_path = os.path.join(output_folder, output_pdb_name)
  pose.dump_pdb(output_path)

Streaming output truncated to the last 5000 lines.
protocols.moves.MonteCarlo: MonteCarlo:: last_accepted_score,lowest_score: -1156.51 -1157.41
protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER FinalMinimizer - final=======================
protocols.ligand_docking.ligand_options.FinalMinimizer: Energy prior to minimizing: 2398.21
protocols.ligand_docking.ligand_options.FinalMinimizer: Energy after minimizing: -1147.4
protocols.rosetta_scripts.ParsedProtocol: setting status to success
core.import_pose.import_pose: File '/content/google_drive/MyDrive/IGLU_SNFR_STRUCTURES/LIGAND_PROTEIN_COMPLEX/relaxed_nir_iglu_BLA.pdb' automatically determined to be of type PDB
core.conformation.Conformation: Found disulfide between residues 84 209
protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER Transform - transform=======================
protocols.qsar.scoring_grid.GridManager: Found a conformation matching hash: 895d839b7f6327d559481830ccdb6f80e1